In [ ]:
from pymc import DiscreteUniform, Exponential, deterministic, Poisson, Uniform
import numpy as np
import pandas as pd
import pymc
import seaborn as sns
import scipy, scipy.stats
from scipy.stats import mode
import pylab
from matplotlib import pyplot as plt
from pymc.Matplot import plot as mcplot
sns.set(color_codes=True)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_csv('data/BRCA-filtered.txt.gz', sep = '\t', index_col = 0)

In [ ]:
dft = df.transpose()

In [ ]:
dft.describe()

In [ ]:
sample_row = 'ZNF114|163071'
sample_row_2 = 'ZNF205|7755'

In [ ]:
# 1 - tumor
# 11 - healthy
healthy = dft.iloc[dft.index.str.contains("-11A-")]
tumor = dft.iloc[dft.index.str.contains("-01A-")]

In [ ]:
plt.figure(figsize=(18, 8))
sns.distplot(healthy[sample_row], label = 'healthy', color = 'green')
sns.distplot(tumor[sample_row], label = 'tumor', color = 'darkred')
plt.legend()

In [ ]:
print 'tumor:', mode(tumor[sample_row]).mode[0]
print 'healthy:', mode(healthy[sample_row]).mode[0]

# Tumor

In [ ]:
successes_t = pymc.DiscreteUniform('successes', lower = 1, upper = 20)

p_successes_t = pymc.Uniform('p_success', 0, 1)

@deterministic()
def mu_t(p = p_successes_t, s = successes_t):
    return successes_t / p_successes_t - successes_t

obs_t = pymc.NegativeBinomial('tumor', mu = mu_t, alpha = successes_t, value = tumor[sample_row], observed = True)

In [ ]:
model_t = pymc.Model([successes_t, mu_t, obs_t])

map_t = pymc.MAP(model_t)
map_t.fit()

mcmc_t = pymc.MCMC(model_t)
mcmc_t.sample(100000, 50000)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (18, 5))

successes_t_trace = mcmc_t.trace('successes')[:]
p_success_t_trace = mcmc_t.trace('p_success')[:]

sns.distplot(successes_t_trace, ax = axs[0], label = 'successes', color = 'red')
sns.distplot(p_success_t_trace, ax = axs[1], label = 'p(success)', color = 'red')

plt.legend()

print 'Successes mean probability:', np.mean(p_success_t_trace)
print 'Successes mean count:', np.mean(successes_t_trace)

In [ ]:
plt.figure(figsize=(10, 5))
sns.kdeplot(tumor[sample_row], shade = False, color = 'red')

n = np.mean(successes_t_trace)
p = np.mean(p_success_t_trace)

print n, p

x = np.arange(-10, 250, 1)
pmf = scipy.stats.nbinom.pmf(x, n, p)
pylab.plot(x, pmf, label = 'mcmc', color = 'orange')
plt.legend()

# Healthy

In [ ]:
successes_h = pymc.DiscreteUniform('successes', lower = 1, upper = 20)

p_successes_h = pymc.Uniform('p_success', 0, 1)

@deterministic()
def mu_h(p = p_successes_h, s = successes_h):
    return successes_h / p_successes_h - successes_h

obs_h = pymc.NegativeBinomial('healthy', mu = mu_h, alpha = successes_h, value = healthy[sample_row], observed = True)

In [ ]:
model_h = pymc.Model([successes_h, mu_h, obs_h])

map_h = pymc.MAP(model_h)
map_h.fit()

mcmc_h = pymc.MCMC(model_h)
mcmc_h.sample(1000000, 50000)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (18, 5))

successes_h_trace = mcmc_h.trace('successes')[:]
p_success_h_trace = mcmc_h.trace('p_success')[:]

sns.distplot(successes_h_trace, ax = axs[0], label = 'successes', color = 'green')
sns.distplot(p_success_h_trace, ax = axs[1], label = 'p(success)', color = 'green')

plt.legend()

print 'Successes mean probability:', np.mean(p_success_h_trace)
print 'Successes mean count:', np.mean(successes_h_trace)

In [ ]:
plt.figure(figsize=(10, 5))
sns.kdeplot(healthy[sample_row], shade = False, color = 'green')

n = np.mean(successes_h_trace)
p = np.mean(p_success_h_trace)

print n, p

x = np.arange(-10, 250, 1)
pmf = scipy.stats.nbinom.pmf(x, n, p)
pylab.plot(x, pmf, label = 'mcmc', color = 'orange')
plt.legend()

In [ ]:
mcplot(mcmc_t.trace('successes'), common_scale=False)
mcplot(mcmc_t.trace('p_success'), common_scale=False)
mcplot(mcmc_h.trace('successes'), common_scale=False)
mcplot(mcmc_h.trace('p_success'), common_scale=False)